In [1]:
import os
import numpy as np
import zipfile
import io
import inspect
import cv2
from sklearn.metrics.pairwise import cosine_distances

from openvino.inference_engine import IECore
from openvino.runtime import Core

In [2]:
core = Core()

In [3]:
devices = core.available_devices

for device in devices:
    device_name = core.get_property(device, "FULL_DEVICE_NAME")
    print(f"{device}: {device_name}")

CPU:        Intel(R) Core(TM) i5-2410M CPU @ 2.30GHz


In [4]:
# models = os.path.join('models', os.listdir('models')[1])

In [5]:
classification_model_xml = 'models/image-retrieval-0001/FP32/image-retrieval-0001.xml'
# classification_model_xml = 'models/vehicle-detection-0200/FP32/vehicle-detection-0200.xml'

model = core.read_model(model=classification_model_xml)
compiled_model = core.compile_model(model=model, device_name="CPU")

In [6]:
input_layer = compiled_model.input(0)
output_layer = compiled_model.output(0)

In [7]:
print(f"input precision: {input_layer.element_type}")
print(f"input shape: {input_layer.shape}")

input precision: <Type: 'float32'>
input shape: [1,224,224,3]


In [8]:
def preprocess_image(image_path, h, w):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (w, h))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype(np.float32)  # Convert to float32
#     image = image.transpose((0, 3, 1, 2))  # Interchange dimensions to [1, 3, 224, 224]
    image = np.array(image)
#     print(image)
    blob = np.expand_dims(image, axis=0)  # Add batch dimension
#     print(blob.shape)
#     blob = blob.transpose((0, 3, 1, 2))
    return blob
    

In [9]:
query_image_path = os.path.join('uploads/images', os.listdir('uploads/images')[0]) # возвращаем путь к картинке по которой ищем совпадение
# preprocessed_query_image = preprocess_image(query_image_path, 224, 224)
preprocessed_query_image = preprocess_image(query_image_path, 224, 224)

In [10]:
result_infer = compiled_model([preprocessed_query_image])[output_layer] # обрабатываем картинку
result_index = np.argmax(result_infer)

In [11]:
# Preprocess the batch of images
batch_images_path = 'archive/data_set_animals.zip'
batch_images_dir = 'uploads/unpack_archive'

with zipfile.ZipFile(batch_images_path, 'r') as zip_ref:
    zip_ref.extractall(batch_images_dir)
    zip_ref.close()

batch_images = []
for image_file in os.listdir(batch_images_dir):
    image_path = os.path.join(batch_images_dir, image_file)
    preprocessed_image = preprocess_image(image_path, 224, 224)
    batch_images.append(preprocessed_image)

batch_images = np.array(batch_images)

result_batch = []
for i in range(len(batch_images)):
    result = compiled_model([batch_images[i]])[output_layer]
    result_batch.append(result)
    
result_batch = np.array(result_batch)

In [12]:
index_box = []
for i in range(len(result_batch)):
    index = np.argmax(result_batch[i])
    index_box.append(index)

In [13]:
comparation = {}

In [14]:
for i in range(len(result_batch)):
    similarities = 1 - cosine_distances(result_infer, result_batch[i])
    percentages = similarities * 100
    comparation[os.listdir(batch_images_dir)[i]] = percentages[0][0]

In [16]:
best_similarity = max(comparation, key= lambda x: comparation[x]) # находим максимальное значение
print(f"Best similarity is : {best_similarity}", comparation[best_similarity], '%') # функция .item() извлекает данные из nd.array

Best similarity is : monkey_rgb.jpg 64.86789 %


In [17]:
comparation

{'bear_bw.jpg': 32.66827,
 'bear_real.jpg': 40.377884,
 'bear_real_2.0.jpg': 4.6197414,
 'bear_real_3.0.jpg': 9.756327,
 'bear_real_4.0.jpg': 1.2092471,
 'bear_rgb.jpg': 5.958337,
 'croc_bw.jpg': 46.84019,
 'croc_real.jpg': -6.2015533,
 'croc_rgb.jpg': -3.2173753,
 'croc_rgb_2.0.jpg': 5.526799,
 'eleph_bw.jpg': -3.8365364,
 'eleph_real.jpg': 0.2820611,
 'eleph_real_2.0.jpg': 11.172581,
 'eleph_rgb.jpg': -8.473719,
 'lion_bw.jpg': 18.487406,
 'lion_real.jpg': -13.319182,
 'lion_rgb.jpg': 9.3071995,
 'monkey_bw.jpg': 43.620605,
 'monkey_bw_2.0.jpg': 2.9915333,
 'monkey_real.jpg': 1.9147813,
 'monkey_real_2.0.jpg': -2.300465,
 'monkey_rgb.jpg': 64.86789,
 'monkey_rgb_2.0.jpg': 14.356577,
 'pig_bw.jpg': 49.382412,
 'pig_bw_2.0.jpg': 55.58306,
 'pig_real.jpg': -7.07798,
 'pig_rgb.jpg': -3.9084196,
 'pig_rgb_2.0.jpg': -1.1155367,
 'tiger_bw.jpg': 30.66666,
 'tiger_bw_2.0.jpg': 39.777176,
 'tiger_real.jpg': -0.30314922,
 'tiger_rgb.jpg': 21.09,
 'tiger_rgb_2.0.jpg': 18.249887}

In [18]:
comp = {'bear_bw.jpg': 0.50496274, 'bear_real.jpg': 0.43372342, 'bear_real_2.0.jpg': 0.39378983, 'bear_real_3.0.jpg': 0.40396994, 'bear_real_4.0.jpg': 0.4154652, 'bear_rgb.jpg': 0.4845882, 'croc_bw.jpg': 0.5385465, 'croc_real.jpg': 0.44251496, 'croc_rgb.jpg': 0.532655, 'croc_rgb_2.0.jpg': 0.5355294, 'eleph_bw.jpg': 0.44278064, 'eleph_real.jpg': 0.44233766, 'eleph_real_2.0.jpg': 0.52261055, 'eleph_rgb.jpg': 0.5083694, 'lion_bw.jpg': 0.47341582, 'lion_real.jpg': 0.43450722, 'lion_rgb.jpg': 0.4624901, 'monkey_bw.jpg': 0.47415784, 'monkey_bw_2.0.jpg': 0.4488975, 'monkey_real.jpg': 0.41878173, 'monkey_real_2.0.jpg': 0.4007879, 'monkey_rgb.jpg': 0.4351797, 'monkey_rgb_2.0.jpg': 0.44396755, 'pig_bw.jpg': 0.44886768, 'pig_bw_2.0.jpg': 0.44045845, 'pig_real.jpg': 0.40686652, 'pig_rgb.jpg': 0.48215085, 'pig_rgb_2.0.jpg': 0.47650814, 'tiger_bw.jpg': 0.5088632, 'tiger_bw_2.0.jpg': 0.409758, 'tiger_real.jpg': 0.38400716, 'tiger_rgb.jpg': 0.4988936, 'tiger_rgb_2.0.jpg': 0.5433155}

In [19]:
comp

{'bear_bw.jpg': 0.50496274,
 'bear_real.jpg': 0.43372342,
 'bear_real_2.0.jpg': 0.39378983,
 'bear_real_3.0.jpg': 0.40396994,
 'bear_real_4.0.jpg': 0.4154652,
 'bear_rgb.jpg': 0.4845882,
 'croc_bw.jpg': 0.5385465,
 'croc_real.jpg': 0.44251496,
 'croc_rgb.jpg': 0.532655,
 'croc_rgb_2.0.jpg': 0.5355294,
 'eleph_bw.jpg': 0.44278064,
 'eleph_real.jpg': 0.44233766,
 'eleph_real_2.0.jpg': 0.52261055,
 'eleph_rgb.jpg': 0.5083694,
 'lion_bw.jpg': 0.47341582,
 'lion_real.jpg': 0.43450722,
 'lion_rgb.jpg': 0.4624901,
 'monkey_bw.jpg': 0.47415784,
 'monkey_bw_2.0.jpg': 0.4488975,
 'monkey_real.jpg': 0.41878173,
 'monkey_real_2.0.jpg': 0.4007879,
 'monkey_rgb.jpg': 0.4351797,
 'monkey_rgb_2.0.jpg': 0.44396755,
 'pig_bw.jpg': 0.44886768,
 'pig_bw_2.0.jpg': 0.44045845,
 'pig_real.jpg': 0.40686652,
 'pig_rgb.jpg': 0.48215085,
 'pig_rgb_2.0.jpg': 0.47650814,
 'tiger_bw.jpg': 0.5088632,
 'tiger_bw_2.0.jpg': 0.409758,
 'tiger_real.jpg': 0.38400716,
 'tiger_rgb.jpg': 0.4988936,
 'tiger_rgb_2.0.jpg': 0.54